# Introduction to Pysyft

Pysyft is an extension to major deep learning toolkits (here focus on pytorch). Right now version 0.3.0 is going on through some kind of an identity crisis with pysyft duet. So I am gonna focus on pysyft v0.2.9.

Officially, Udacity Secure and Private AI course supports syft==0.1.2a1 , but I am gonna run on 0.2.9 and see if I can adjust to the later version.

In [ ]:
!pip install syft==0.2.9 >/dev/null

ERROR: tensorflow 2.4.1 has requirement numpy~=1.19.2, but you'll have numpy 1.18.5 which is incompatible.
ERROR: google-colab 1.0.0 has requirement notebook~=5.3.0; python_version >= "3.0", but you'll have notebook 5.7.8 which is incompatible.
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.22.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement tornado~=5.1.0; python_version >= "3.0", but you'll have tornado 4.5.3 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: bokeh 2.1.1 has requirement tornado>=5.1, but you'll have tornado 4.5.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


In [ ]:
import torch
import syft as sy

In [ ]:

x = torch.tensor([1,2,3,4,5])
y = x+x

print(y)

hook = sy.TorchHook(torch)

torch.tensor([2,3,4,5,6])

tensor([ 2,  4,  6,  8, 10])


tensor([2, 3, 4, 5, 6])

## Basic Remote Execution in PySyft
Instead of torch tensors we are gonna create pointers to tensors. 

First create a pretend machine owned by a pretend person say, Bob.

In [ ]:
bob = sy.VirtualWorker(hook, id='bob')

In [ ]:
# Now it is empty
bob._objects

{}

In [ ]:
x = torch.tensor([1,2,3,4,5])
# Sending the tensor to bob
x = x.send(bob) # If initially this throws error, restart runtime and run again... Weird.

bob._objects

{98010569556: tensor([1, 2, 3, 4, 5])}

What was returned when I did x.send(bob)?

Now, x.send() will return a pointer to a remote object. Pointer is a tensor and has all the tensor api's with it.

However, when you do some tensor op on it, each op is serialized into something internal like json and then sent to bob. Then the virtual worker bob will execute those ops and then returns the pointer to the result.

In [ ]:
x

(Wrapper)>[PointerTensor | me:67098642563 -> bob:98010569556]

In [ ]:
x.id

67098642563

In [ ]:
x.id_at_location

98010569556

In [ ]:
x.owner

<VirtualWorker id:me #objects:0>

In [ ]:
x.location

<VirtualWorker id:bob #objects:1>

Owner 'me' is the local worker that was created when we hooked :)

So now all commands are done on the device 'me'. AKA local worker.

In [ ]:
x # x is a pointer

(Wrapper)>[PointerTensor | me:67098642563 -> bob:98010569556]

In [ ]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [ ]:
bob._objects

{}

In [ ]:
# Now bob does not contain any object anymore.
# So another x.get() will give you this error
x.get()

AttributeError: ignored

## Multipe workers at a time

In [ ]:
# We already have bob with {}
alice = sy.VirtualWorker(hook, id='alice')
x = torch.tensor([2,3,4,5,6])
x_ptr = x.send(bob, alice)
x_ptr

(Wrapper)>[MultiPointerTensor]
	-> [PointerTensor | me:49779696097 -> bob:74300958979]
	-> [PointerTensor | me:55541863964 -> alice:74300958979]

A pointer that points to multiple machines is returned.

In [ ]:
x_ptr.get()

[tensor([2, 3, 4, 5, 6]), tensor([2, 3, 4, 5, 6])]

This returns two tensors.

In [ ]:
x = torch.tensor([1,2,3,4,5]).send(bob, alice)
x.get(sum_results=True)

tensor([ 2,  4,  6,  8, 10])

## Remote Arithmetic

In [ ]:
x = torch.tensor([1,2,3,4,5]).send(bob)
y = torch.tensor([1,1,1,1,1]).send(bob)

In [ ]:
x

(Wrapper)>[PointerTensor | me:83170498742 -> bob:89751322106]

In [ ]:
y

(Wrapper)>[PointerTensor | me:54981061737 -> bob:5489897619]

The best thing about these pointer tensors is that we can pretend that they are just regular tensors. 

In [ ]:
z = x + y # this will be executed at bob's machine
z

(Wrapper)>[PointerTensor | me:71162559070 -> bob:76707922489]

In [ ]:
z.location

<VirtualWorker id:bob #objects:3>

In [ ]:
z = z.get(); z

tensor([2, 3, 4, 5, 6])

In [ ]:
bob._objects

{5489897619: tensor([1, 1, 1, 1, 1]), 89751322106: tensor([1, 2, 3, 4, 5])}

In [ ]:
z = torch.add(x, y); z # Again executed at bob's machine

(Wrapper)>[PointerTensor | me:78364046656 -> bob:71043509546]

In [ ]:
bob._objects

{5489897619: tensor([1, 1, 1, 1, 1]),
 71043509546: tensor([2, 3, 4, 5, 6]),
 89751322106: tensor([1, 2, 3, 4, 5])}

In [ ]:
z = z.get(); z

tensor([2, 3, 4, 5, 6])

### This simple arithmetic can be extended to the full pytorch functionality with LA and calculus for NN

In [ ]:
x = torch.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = torch.tensor([1,1,1.,1,1], requires_grad=True).send(bob)
z = (x + y).sum()
z.backward()
x = x.get(); x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [ ]:
x.grad

tensor([1., 1., 1., 1., 1.])